# Cellpose in Python

<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../../notebooks/05_segmentation/deep_learning/cellpose_notebook.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/05_segmentation/deep_learning/cellpose_colab.ipynb" target="_blank">
        <img class="button-icon" src="../../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "matplotlib",
#     "tifffile",
#     "cellpose"
# ]
# ///

## Overview

[Website](https://www.cellpose.org) | [GitHub](https://github.com/mouseland/cellpose) | [Paper](https://www.biorxiv.org/content/10.1101/2025.04.28.651001v1)

In this section...TO COMPLETE

If you do not have an Apple Silicon Mac or a GPU we suggest to run this notebook in Google Colab. 

The images we will use for this section can be downloaded from the <a href="../../../_static/data/05_segmentation_cellpose.zip"> <i class="fas fa-download"></i> Cellpose Dataset</a>.

## Import Libraries

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import tifffile
from cellpose import core, io, models, plot
from tqdm import tqdm



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	darwin 
python version: 	3.13.0 
torch version:  	2.7.1! The neural network component of
CPSAM is much larger than in previous versions and CPU execution is slow. 
We encourage users to use GPU/MPS if available. 




## Setup

In [ ]:
io.logger_setup()  # to get printing of progress

use_gpu = core.use_gpu()
print("GPU available:", use_gpu)

## Run Cellpose on A Sample Image

In this section we will...

### Load the Image

In [ ]:
image_path = "../../../_static/images/cellpose/cell_cellpose.tif"
image = tifffile.imread(image_path)

print(image.shape)

### Initialize the Model

To initialize Cellpose model we can use the [`models.CellposeModel()`](https://cellpose.readthedocs.io/en/latest/api.html#cellposemodel) class.

Among the parameters we can set if we want to use `gpu` (if available) as well which **model** to use. The current default is Cellpose-SAM as `cpsam` (another example is `cyto3`, the previous U-Net-based model).

If it is the first time you run this notebook, the model will be downloaded automatically. This may take a while.

In [ ]:
model = models.CellposeModel(gpu=use_gpu, model_type="cpsam")

### Run Cellpose

After initializing the model, we can run it on the image using the [`model.eval()`](https://cellpose.readthedocs.io/en/latest/api.html#id0) method.

Here are some of the parameters you can change:

* ***flow_threshold*** is  the  maximum  allowed  error  of  the  flows  for  each  mask.   The  default  is 0.4.
    *  **Increase** this threshold if cellpose is not returning as many masks as you’d expect (or turn off completely with 0.0)
    *   **Decrease** this threshold if cellpose is returning too many ill-shaped masks.

* ***cellprob_threshold*** determines proability that a detected object is a cell.   The  default  is 0.0.
    *   **Decrease** this threshold if cellpose is not returning as many masks as you’d expect or if masks are too small
    *   **Increase** this threshold if cellpose is returning too many masks esp from dull/dim areas.

* ***tile_norm_blocksize*** determines the size of blocks used for normalizing the image. The default is 0, which means the entire image is normalized together.
  You may want to change this to 100-200 pixels if you have very inhomogeneous brightness across your image.

In [ ]:
channel = 0  # channel to use for cell detection, 0 cytoplasm, 1 nucleus

# Cellpose parameters
flow_threshold = 0.4
cellprob_threshold = 0.0
tile_norm_blocksize = 0

masks, flows, styles = model.eval(
    image[channel],
    batch_size=32,
    flow_threshold=flow_threshold,
    cellprob_threshold=cellprob_threshold,
    normalize={"tile_norm_blocksize": tile_norm_blocksize},
)

### Display the Results

In [ ]:
fig = plt.figure(figsize=(12, 5))
plot.show_segmentation(fig, image[channel], masks, flows[0])
plt.tight_layout()
# Optional if you want to also save the figure
# plt.savefig(f"path/to/output/{Path(image_path).stem}_cp_output.png")
plt.show()

<div align="left"> <img src="https://raw.githubusercontent.com/HMS-IAC/bobiac/main/_static/images/cellpose/cellpose_out.png" alt="Ilastik Logo" width="1000"></div>

## Run Cellpose on a Folder of Images

...

In [38]:
# folder_path = Path("data/05_segmentation_cellpose")
folder_path = Path("/Users/fdrgsp/Desktop/05_segmentation_cellpose")

# Read file names
images = []
for f in folder_path.glob("*.tif"):
    images.append(f)
# same as running: files = [f for f in folder_path.glob("*.tif")]

# Cellpose parameters
flow_threshold = 0.4
cellprob_threshold = 0.0
tile_norm_blocksize = 0

# Run Cellpose on all images
for image_path in tqdm(images):
    image = tifffile.imread(image_path)
    masks, flows, styles = model.eval(
        image,
        batch_size=32,
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        normalize={"tile_norm_blocksize": tile_norm_blocksize},
    )

    # Optional: display each image with its segmentation
    # fig = plt.figure(figsize=(12, 5))
    # plot.show_segmentation(fig, image, masks, flows[0])
    # plt.tight_layout()
    # plt.show()

    # Save the segmentation results
    output_path = folder_path / f"{image_path.stem}_labeled_mask.tif"
    tifffile.imwrite(output_path, masks.astype("uint16"))